In [1]:
import sqlite3
import random

# Abrir base de datos
Primero es necesario abrir nuestra base de datos y definir el cursor.

In [2]:
connection = sqlite3.connect('AC09.db') 
cursor = connection.cursor()   

La primera misión es agregar las columnas Precio y Distancia. Para esto utilizamos lo indicado en el material de clases `02 - SQL`.
> Finalmente, si se busca modificar la estructura de una tabla, es posible hacerlo mediante el comando ALTER:

> ALTER TABLE Estudiantes ADD COLUMN carrera VARCHAR(30)

En nuestro caso, hay que cambiar `Estudiantes` por `Flights`, `carrera` por `Precio` o `Distancia` y `VARCHAR(30)` por `INTEGER`.

In [3]:
cursor.execute("ALTER TABLE Flights ADD COLUMN Precio INTEGER")
cursor.execute("ALTER TABLE Flights ADD COLUMN Distancia INTEGER")
connection.commit()
# Si aparece un OperationalError es que las tablas ya tienen esas columnas

Para validar que existen, usamos el comando que nos enseñaron `PRAGMA` para ver la información de las columnas.

In [4]:
cursor.execute('PRAGMA table_info([Flights])')
cursor.fetchall()

[(0, 'Airline', 'INTEGER', 0, None, 0),
 (1, 'FlightNo', 'INTEGER', 0, None, 0),
 (2, 'SourceAirport', 'varchar(3)', 0, None, 0),
 (3, 'DestAirport', 'varchar(3)', 0, None, 0),
 (4, 'Precio', 'INTEGER', 0, None, 0),
 (5, 'Distancia', 'INTEGER', 0, None, 0)]

Ahora que ya tenemos nuestras 2 columnas creadas, creamos dos funciones encargada de generar precios y distancia random.

-------------------------

Para generar precios random, tenemos que iterar sobre los datos. Disponemos de dos opciones

La primera es utilizar lo enseñado en el material:

>Finalmente, con el fin de hacer más limpia (y formateable) la impresión de los resultados obtenidos por una consulta, es posible utilizar el cursor resultante de una consulta como un iterador:

>  
```
for country in cursor.execute('SELECT * FROM countries ORDER BY name'):
    print(country)
```

El problema de lo anterior esque solo se puede hacer si **dentro del for no se realizar ningún `cursor.execute()`**. En este caso queremos iterar sobre el resultado para editar la tabla, es decir, si tendremos un `cursor.execute()` dentro del for y por ello, recurrimos a la segunda forma, la cual es utilizar `fetchall()` para capturar el resultado e iterar sobre ese. Más adelante usaremos la forma 1.

In [5]:
def generar_precio_random(cursos, connection):
    cursor.execute("Select SourceAirport, DestAirport from Flights")
    # fetchall nos traerá todas las filas incluyendo el header
    # Por tal motivo le agregamos un `[1:]` para decirle que parta desde el segundo elemento
    # en adelante (considerar el primer elemento como la posición 0)
    tuplas = cursor.fetchall()[1:]

    for tupla in tuplas: # Recorremos cada fila
        precio = random.randint(1000, 7000)
        SourceAirport = tupla[0]
        DestAirport = tupla[1]
        cursos.execute("UPDATE Flights SET Precio = ? WHERE SourceAirport = ? AND DestAirport = ?",
                       [precio, SourceAirport, DestAirport])
        
    # Guardamos los cambios
    connection.commit()

In [6]:
generar_precio_random(cursor, connection)
# Verificamos si funcionó
cursor.execute("Select * from Flights")
cursor.fetchall() [:15] # El [:15] es para ver nomás 15 filas

[('Airline', 'FlightNo', 'SourceAirport', 'DestAirport', None, None),
 (1, 28, 'APG', 'ASY', 1863, None),
 (1, 29, 'ASY', 'APG', 1454, None),
 (1, 44, 'CVO', 'ACV', 4862, None),
 (1, 45, 'ACV', 'CVO', 4900, None),
 (1, 54, 'AHD', 'AHT', 2223, None),
 (1, 55, 'AHT', 'AHD', 3930, None),
 (1, 90, 'ATO', 'ABR', 2609, None),
 (1, 91, 'ABR', 'ATO', 4227, None),
 (1, 108, 'ANV', 'MMI', 3094, None),
 (1, 109, 'MMI', 'ANV', 3184, None),
 (1, 142, 'ALI', 'AKC', 6889, None),
 (1, 143, 'AKC', 'ALI', 6900, None),
 (1, 148, 'HMN', 'ABL', 1344, None),
 (1, 149, 'ABL', 'HMN', 5291, None)]

Ahora haremos lo mismo para la función de distancia, pero realizaremos 2 cambios
1. Utilizaremos `format` para hacer las consultas
2. Hay que verificar que la distancia entre A y B sea la misma entre B y A.

In [7]:
def generar_distancia_random(cursos, connection):
    cursor.execute("Select SourceAirport, DestAirport from Flights")
    # fetchall nos traerá todas las filas incluyendo el header
    # Por tal motivo le agregamos un `[1:]` para decirle que parta desde el segundo elemento
    # en adelante (considerar el primer elemento como la posición 0)
    tuplas = cursor.fetchall()[1:]
    
    # Aquí dejamos los espacios con {} libres para despues llenarlos con format
    # En este caso debemos agregar comillas a los string como el SourceAirport y DestAirport
    consulta = "UPDATE Flights SET Distancia = {} WHERE SourceAirport = '{}' AND DestAirport = '{}'"
    
    for tupla in tuplas: # Recorremos cada fila
        distancia = random.randint(1000, 7000)
        SourceAirport = tupla[0]
        DestAirport = tupla[1]
        cursos.execute(consulta.format(distancia, SourceAirport, DestAirport))
        
        # Aquí vemos si existe la ruta inversa y sí existe, le ponemos la misma distancia 
        if (DestAirport, SourceAirport) in tuplas:
            cursos.execute(consulta.format(distancia, DestAirport, SourceAirport))
            
    # Guardamos los cambios
    connection.commit()

In [8]:
generar_distancia_random(cursor, connection)
# Verificamos si funcionó
cursor.execute("Select * from Flights")
cursor.fetchall()[:15] # El [:15] es para ver nomás 15 filas

[('Airline', 'FlightNo', 'SourceAirport', 'DestAirport', None, None),
 (1, 28, 'APG', 'ASY', 1863, 2871),
 (1, 29, 'ASY', 'APG', 1454, 2871),
 (1, 44, 'CVO', 'ACV', 4862, 1461),
 (1, 45, 'ACV', 'CVO', 4900, 1461),
 (1, 54, 'AHD', 'AHT', 2223, 6282),
 (1, 55, 'AHT', 'AHD', 3930, 6282),
 (1, 90, 'ATO', 'ABR', 2609, 4188),
 (1, 91, 'ABR', 'ATO', 4227, 4188),
 (1, 108, 'ANV', 'MMI', 3094, 1701),
 (1, 109, 'MMI', 'ANV', 3184, 1701),
 (1, 142, 'ALI', 'AKC', 6889, 3753),
 (1, 143, 'AKC', 'ALI', 6900, 3753),
 (1, 148, 'HMN', 'ABL', 1344, 1326),
 (1, 149, 'ABL', 'HMN', 5291, 1326)]

# Consultas
----------
### Consulta 1:
> Daniela se encuentra en Apalachicola y un benefactor anónimo decide costearle solo un pasaje desde
allí a cualquier parte, para que siga buscando a Vicente. Si ella decidiera darse la gran vida, invertiría
la donación en un costoso viaje. ¿Cuál es el vuelo más lujoso al que puede optar desde allí?

En resumen, debemos buscar el viaje más costoso desde Apalachicola. Para esto primero necesitamos la sigla de Apalachicola, la cual se obtiene de `Airlines` y obtener la fila que tenga el máximo costo

Aquí aprovecharemos lo visto en el material de hacer un for al `cursos.execute` para itelar sobre el resultado

In [9]:
ruta_lujosa = None
precio_maximo = 0
for ruta in cursor.execute("Select F.Airline, F.FlightNo, F.SourceAirport, F.DestAirport, F.Distancia, F.Precio, A.City \
                            FROM Flights F, Airports A \
                            WHERE F.SourceAirport = A.AirportCode AND A.City = 'Apalachicola'"):
    
    precio = ruta[5]
    if precio >= precio_maximo:
        ruta_lujosa = ruta
        precio_maximo = precio

print("Airline, FlightNo, SourceAirport, DestAirport, Distancia, Precio, Ciudad")
ruta_lujosa

Airline, FlightNo, SourceAirport, DestAirport, Distancia, Precio, Ciudad


(11, 591, 'AAF', 'ANY', 2063, 6282, 'Apalachicola')

### Consulta 2:
> Es probable que Vicente haya escogido vivir en un aeropuerto al que no lleguen muchos vuelos. ¿Cuál
es el aeropuerto al que llegan menos rutas?

En resumen, debemos agrupar los aeropuertos por el DestAirport para saber cuantas rutas llegan a ese aeropuerto y luego contar. Finalmente ver cual es el DestAirport

In [10]:
ruta_minima = None
cantidad_ruta = float("+inf") #Numero extremadamente grande
for ruta in cursor.execute("Select DestAirport, count(*) FROM Flights GROUP BY DestAirport"):
    if ruta[0] != 'DestAirport': #Analizamos las rutas que no contengan el header
        cantidad_rutas_llegadas = ruta[1]
        if cantidad_rutas_llegadas <= cantidad_ruta:
            ruta_minima = ruta
            cantidad_ruta = cantidad_rutas_llegadas

print("El aeropuerto es {} con {} rutas".format(ruta_minima[0], ruta_minima[1]))

El aeropuerto es AID con 3 rutas


### Consulta 5:
> Daniela se dio cuenta, finalmente, de que Vicente no pudo haber viajado a más de un aeropuerto, pues
debió haberse gastado todo su dinero en comida después del primer viaje. Además, Daniela sabe que
Vicente quería alejarse lo más posible de la UC. Asumiendo que la UC está en Apalachicola, ¿dónde
está Vicente?

Tenemos que buscar la ruta más lejana partiendo de apalachicola. Notar que es lo mismo que la consulta 1 pero no buscaremos el precio máximo, sino que la distancia máxima. Una opción valida es recurrir al `copy/paste` de esa solución y cambiar algunos detalles. La otra opción (que se pude haber hecho antes) es ordenar los datos por distancia y sacar el primero. En este caso usaremos ambas

In [11]:
# Forma 1 Copy/Paste y edición
cursor.execute("Select F.Airline, F.FlightNo, F.SourceAirport, F.DestAirport, F.Distancia, F.Precio, A.City \
                FROM Flights F, Airports A \
                WHERE F.SourceAirport = A.AirportCode AND A.City = 'Apalachicola'")
ruta_lejana = None
distancia_maxima = 0
for ruta in cursor.fetchall():
    distancia = ruta[4]
    if distancia >= distancia_maxima:
        ruta_lejana = ruta
        distancia_maxima = distancia

print("Airline, FlightNo, SourceAirport, DestAirport, Distancia, Precio, Ciudad")
ruta_lejana

Airline, FlightNo, SourceAirport, DestAirport, Distancia, Precio, Ciudad


(4, 166, 'AAF', 'AKI', 6912, 2462, 'Apalachicola')

In [12]:
# Forma 2 Usar ORDER BY
cursor.execute("Select F.Airline, F.FlightNo, F.SourceAirport, F.DestAirport, F.Distancia, F.Precio, A.City \
                FROM Flights F, Airports A \
                WHERE F.SourceAirport = A.AirportCode AND A.City = 'Apalachicola'\
                ORDER BY Distancia DESC")

print("Airline, FlightNo, SourceAirport, DestAirport, Distancia, Precio, Ciudad")
cursor.fetchone()

Airline, FlightNo, SourceAirport, DestAirport, Distancia, Precio, Ciudad


(4, 166, 'AAF', 'AKI', 6912, 2462, 'Apalachicola')

### Consulta 6:
> Después de ir a buscar a Vicente al aeropuerto que determinaste, Daniela debe decidir a qué paraíso
terrenal ir. Para esto deben considerar su presupuesto, que es equivalente a 3 veces el precio promedio
de los vuelos de la aerolínea con presencia en la mayor cantidad de ciudades. ¿Cuál es su presupuesto?$^{3}$

$^{3}$ Una aereolínea está presente en una ciudad si alguno de sus vuelos tiene como origen o destino a esa ciudad.


Primero necesitamos encontrar la aerolínea con mayor presencia y luego obtener el precio promedio de todos sus vuelos.
Para la primera parte es clave lograr asociar cada aerolinea a las diferentes ciudades que llegan o salen.

La solución propuesta es hacer una consulta que asocie cada aerolínea a la ciudad que salen y otra consulta que asocie cada aerolinea a la ciudad que entran. Esto sería algo de la forma:

|Aerolínea | Ciudad_que_sale|
|--------|-----------|
|1|ciudad_1|
|1|ciudad_2|
|...| ...|


|Aerolínea | Ciudad_que_entra|
|--------|-----------|
|5|ciudad_3|
|8|ciudad_7|
|...| ...|

In [13]:
cursor.execute("Select F.Airline, A.City\
                FROM Flights F, Airports A \
                WHERE F.SourceAirport = A.AirportCode")

ciudades_origen = cursor.fetchall()

cursor.execute("Select F.Airline, A.City\
                FROM Flights F, Airports A \
                WHERE F.DestAirport = A.AirportCode")

ciudades_destino = cursor.fetchall()

# ¿Que tenemos? 
1. La lista `ciudades_origen` que son tuplas de aerolíneas con la ciudad en la que parte el vuelo.
2. La lista `ciudades_destino` que son tuplas de aerolíneas con la ciudad a la que llega el vuelo.

Ahora usaremos un diccionario donde la `key` será la aerolínea y la `value` será un set de las ciudades en las que interactúa dicha aerolínea. Con este terminaremos con un diccionario donde cada `key` es una aerolínea distinta y su `value` es un set de todos las ciudades relacionadas con dicha aerolínea.

In [14]:
airline_dictio = {}

for airline, ciudad in ciudades_origen:
    if airline in airline_dictio:
        airline_dictio[airline].add(ciudad)
    else:
        airline_dictio[airline] = set()
        
for airline, ciudad in ciudades_destino:
    if airline in airline_dictio:
        airline_dictio[airline].add(ciudad)
    else:
        airline_dictio[airline] = set()

Ahora buscaremos la aerolínea con mayor cantidad de ciudades:

In [15]:
airline = None
ciudades = 0
for airline_id in airline_dictio:
    num_ciudades = len(airline_dictio[airline_id])
    if num_ciudades >= ciudades:
        airline = airline_id
        ciudades = num_ciudades
        
airline

2

Con este ID, buscamos los vuelos de esa aerolínea y sacamos el promedio de dinero.

In [16]:
cursor.execute("Select AVG(Precio)\
                FROM Flights\
                WHERE Airline = ?\
                GROUP BY Airline", [airline])
promedio = cursor.fetchone()
promedio

(4344.11,)

In [17]:
print("El presupuesto de Daniela es 3 veces ese promedio. En este caso es {:.3f}".format(promedio[0]*3))

El presupuesto de Daniela es 3 veces ese promedio. En este caso es 13032.330


Finalmente cerramos la conexion y nos vamos para la casa :D

In [18]:
connection.close()

## **Aclaración**
No es estrictamente necesario usar todo en SQL para encontrar algún dato. La idea es que la consulta principal sea en sql como los `join` y poner los `where` o `group by` y finalmente recorrir a Python. Podran notar que la consulta 5 fue hecha de dos formas totalmente válidas. Una usando python para encontrar el máximo y otro usando el `ORDER BY`